# **GeolifePrj_01_df2gjson.ipynb**
## Convierte los DataFrames creados en **GeolifePrj_00_plt2df** (archivos geolife_xxx.zip) a archivos GeoJson simplificados:
## **geolife_xxx_simpl.geojson** (puntos gps=(lat,lon) y su date="fecha/hora" respectiva)
## **geolife_xxx_lines.geojson** (una linea por trayectoria)
## **geolife_xxx_linesimpl.geojson** (lineas despues de simplificar la data)


In [53]:
import geopandas
import numpy as np
import pandas as pd
#from shapely.geometry import Point
from geojson import  Point,Feature, FeatureCollection, dumps

import csv, json

import seaborn as sns

from shapely.geometry import LineString
from shapely.geometry import Point

from dateutil.parser import parse as parsedt

### def **plt2geojson(...):** Convierte archivo original plt a geojson
### def **simplecsv2geojson(...)** Convierte un archivo **geolife_xxx_simpl.csv** generado por la funcion **crea_csv_simpl():** a un archivo geojson

In [9]:
def plt2geojson(from_plt_file,to_geojson_file):
    features = []
    with open(from_plt_file, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for _ in range(6): next(reader)
        for latitude,longitude,empty,altitude,days,date,time in reader:
            latitude, longitude = map(float, (latitude, longitude))
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = {
                        'altitude': altitude,
                        'date': f"{date} {time}"
                    }
                )
            )

    collection = FeatureCollection(features)
    with open(to_geojson_file, "w") as f:
        f.write(f"{collection}")
        
def simplecsv2geojson(from_csv_file,to_geojson_file):
    features = []
    with open(from_csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for date,latitude,longitude,label in reader:
            latitude, longitude = map(float, (latitude, longitude))
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = {
                        'date': f"{date}",
                        'label': f"{label}"
                    }
                )
            )

    collection = FeatureCollection(features)
    with open(to_geojson_file, "w") as f:
        f.write(f"{collection}")

### def **df2geojson(...)** genera desde **geolife_xxx.zip** un archivo **geolife_xxx.geojson** (TODA LA DATA)

In [10]:
def df2geojson(df,to_geojson_file):
    df=df.astype(str)
    lat = df['lat']
    lng = df['lon']
    #df = df.drop(columns=['lat', 'lon'])
    df = df.drop(columns=['lat', 'lon','user'])    
    
    feat_list = []
    failed = []
    for i in range(0, len(df.index)):
        props = dict(df.iloc[i])
        try:
            f = Feature(geometry=Point((float(lng.iloc[i]),float(lat.iloc[i]))),
                       properties = props)
            feat_list.append(f)
        except Exception as ex:
            print("Exception:",ex)
        
    collection = FeatureCollection(feat_list)
    with open(to_geojson_file, "w") as f:
        f.write(f"{collection}")

## def **crea_simpl_csv(**xxx**):** genera una version simplificada del DataFrame almacenado en **geolife_xxx.zip** y lo guarda como **geolife_xxx_simpl.csv**
### Simplificaciones:
### - *Guarda solo 5 digitos decimales para latitude y longitude*
### - *Bota la columna 'user' y 'altitude'*
### - *Guarda solo maximo UNA coordena gps por minuto (si hay mas de un punto gps por minuto desecha los sobrantes)*

In [14]:
%%time
#creamos una version simplificada en csv
def crea_simpl_csv(xxx):
    usrdf = pd.read_pickle(f'geolife_{xxx:03d}.zip')
    usrdf.columns=['date','latitude','longitude','altitude','label','user']
    #dejamos 5 digitos de precision, que equivale a una resolucion de 8m^2 en el mapa. Explicacion: https://planspace.org/20180719-geolocation_precision_by_digit/
    usrdf['latitude']=usrdf['latitude'].apply(lambda x:f"{x:.5f}")
    usrdf['longitude']=usrdf['longitude'].apply(lambda x:f"{x:.5f}")
    usrdf.drop(columns=['user','altitude'],inplace=True)
    usrdf=usrdf.astype(str)
    #elimina los segundos y luego los repetidos, con lo que solo nos quedamos con una resolucion de 
    #un punto gps por minutos:
    usrdf['date']=usrdf['date'].apply(lambda x:x[:-3]+':00')
    usrdf = usrdf.drop_duplicates(subset='date', keep='first')
    usrdf.to_csv(f"geolife_{xxx:03d}_simpl.csv", index=False)


Wall time: 0 ns


#### Creamos los CSV simplificados

In [ ]:
for xxx in range(0,182):
    crea_simpl_csv(xxx)    
    ## Muestra avance ;) [########>-------------------------------------] 
    print("["+"#"*int(100*(1+xxx)/182)+">"+"-"*int(100*(181-xxx)/181)+"]"+f" {xxx:03d}/182",end="\r")

#### Generamos **geolife_xxx_simpl.geojson** a partir de **geolife_xxx_simple.csv**

In [7]:
%%time
#creamos una version simplificada en geojson
for usuario_num in range(0,182):
    simplecsv2geojson(f"geolife_{usuario_num:03d}_simpl.csv",f"geolife_{usuario_num:03d}_simpl.geojson")
    ## Avance ;)
    print("["+"#"*int(100*(1+usuario_num)/182)+">"+"-"*int(100*(181-usuario_num)/181)+"]"+f" {usuario_num:03d}/182",end="\r")

Wall time: 24.7 s####################################################################################>] 181/182


## Primero generamos
## **geolife_xxx_lines.geojson** (una linea por trayectoria= una linea por día)
## **geolife_xxx_linesimpl.geojson** (igual pero desde version **_simpl** )

### cargamos un **geolife_xxx_simpl.geojson** y mostramos

In [99]:
# generamos geolife_xxx_linesimpl.geojson donde xxx=
xxx=53
archivo_gj=f"geolife_{xxx:03d}_"
archivo_salida_gj = archivo_gj+"linesimpl.geojson"
archivo_gj=archivo_gj+"simpl.geojson"
print(archivo_gj," -> " ,archivo_salida_gj)
gdf = geopandas.read_file(archivo_gj)
gdf.head(5)

geolife_053_simpl.geojson  ->  geolife_053_linesimpl.geojson


,date,label,geometry
0,2008-04-29T08:21:00,0,POINT (116.33038 39.97630)
1,2008-04-29T09:15:00,0,POINT (116.33007 39.97523)
2,2008-04-30T13:39:00,3,POINT (116.33028 39.97857)
3,2008-04-30T13:46:00,3,POINT (116.34682 39.97067)
4,2008-04-30T13:48:00,0,POINT (116.35105 39.96625)


### generamos un objeto **LineString** por cada dia

In [102]:
list_dates=list(gdf.date.str[:10].unique())
gempty= {'date': [], 'geometry': []}
gdf_new=geopandas.GeoDataFrame(gempty, crs="EPSG:4326")
for dia in list_dates[1:]:
    plst=gdf.loc[gdf.date.str[:10]==dia,'geometry'].to_list()
    ls=LineString(plst)
    grow = {'date': [dia], 'geometry': [ls]}
    g1=geopandas.GeoDataFrame(grow, crs="EPSG:4326")
    gdf_new=gdf_new.append(g1)
gdf_new['stroke']="#f71a3a"
gdf_new['stroke-width']=2
gdf_new['stroke-opacity']=0.7

### vemos el **geoDataFrame** producido

In [103]:
gdf_new.head(5)

,date,geometry,stroke,stroke-width,stroke-opacity
0,2008-04-30,"LINESTRING (116.33028 39.97857, 116.34682 39.9...",#f71a3a,2,0.7
0,2008-05-01,"LINESTRING (119.48115 39.81815, 119.48032 39.8...",#f71a3a,2,0.7
0,2008-05-02,"LINESTRING (119.47575 39.81952, 119.47877 39.8...",#f71a3a,2,0.7
0,2008-05-05,"LINESTRING (116.32932 39.97497, 116.33818 39.9...",#f71a3a,2,0.7
0,2008-05-06,"LINESTRING (116.35533 39.96218, 116.35507 39.9...",#f71a3a,2,0.7


### Lo guardamos a **geolife_xxx_linesimpl.geojson**

In [94]:
gdf_new.to_file(archivo_salida_gj, driver="GeoJSON")

## Ahora metemos todo a la función def **simpl2lines(**xxx**):** y la iteramos

In [118]:
def simpl2lines(xxx):
    archivo_gj=f"geolife_{xxx:03d}_"
    archivo_salida_gj = archivo_gj+"linesimpl.geojson"
    archivo_gj=archivo_gj+"simpl.geojson"
    print(archivo_gj," -> " ,archivo_salida_gj)
    gdf = geopandas.read_file(archivo_gj)
    
    list_dates=list(gdf.date.str[:10].unique())
    gempty= {'date': [], 'geometry': []}
    gdf_new=geopandas.GeoDataFrame(gempty, crs="EPSG:4326")
    for dia in list_dates:
        plst=gdf.loc[gdf.date.str[:10]==dia,'geometry'].to_list()
        if len(plst)<2:
            continue
        else:
            ls=LineString(plst)
            grow = {'date': [dia], 'geometry': [ls]}
            g1=geopandas.GeoDataFrame(grow, crs="EPSG:4326")
            gdf_new=gdf_new.append(g1)
    gdf_new['stroke']="#f71a3a"
    gdf_new['stroke-width']=2
    gdf_new['stroke-opacity']=0.7
    gdf_new.to_file(archivo_salida_gj, driver="GeoJSON")

In [ ]:
for i in range(0,182):
    simpl2lines(i)

geolife_000_simpl.geojson  ->  geolife_000_linesimpl.geojson
geolife_001_simpl.geojson  ->  geolife_001_linesimpl.geojson
geolife_002_simpl.geojson  ->  geolife_002_linesimpl.geojson
geolife_003_simpl.geojson  ->  geolife_003_linesimpl.geojson
geolife_004_simpl.geojson  ->  geolife_004_linesimpl.geojson
geolife_005_simpl.geojson  ->  geolife_005_linesimpl.geojson
geolife_006_simpl.geojson  ->  geolife_006_linesimpl.geojson
geolife_007_simpl.geojson  ->  geolife_007_linesimpl.geojson
geolife_008_simpl.geojson  ->  geolife_008_linesimpl.geojson
geolife_009_simpl.geojson  ->  geolife_009_linesimpl.geojson
geolife_010_simpl.geojson  ->  geolife_010_linesimpl.geojson
geolife_011_simpl.geojson  ->  geolife_011_linesimpl.geojson
geolife_012_simpl.geojson  ->  geolife_012_linesimpl.geojson
geolife_013_simpl.geojson  ->  geolife_013_linesimpl.geojson
geolife_014_simpl.geojson  ->  geolife_014_linesimpl.geojson
geolife_015_simpl.geojson  ->  geolife_015_linesimpl.geojson
geolife_016_simpl.geojso

In [ ]:
### **simpl2lines2()** le agrega un marcador donde empieza y donde termina la trayectoria

In [118]:
def simpl2lines2(xxx):
    archivo_gj=f"geolife_{xxx:03d}_"
    archivo_salida_gj = archivo_gj+"linesimpl.geojson"
    archivo_gj=archivo_gj+"simpl.geojson"
    print(archivo_gj," -> " ,archivo_salida_gj)
    gdf = geopandas.read_file(archivo_gj)
    
    gempty= {'date': [], 'geometry': []}
    gdf_new=geopandas.GeoDataFrame(gempty, crs="EPSG:4326")    
    
    list_dates=list(gdf.date.str[:10].unique())
    gempty= {'date': [], 'geometry': []}
    gdf_new=geopandas.GeoDataFrame(gempty, crs="EPSG:4326")
    for dia in list_dates:
        plst=gdf.loc[gdf.date.str[:10]==dia,'geometry'].to_list()
        if len(plst)<2:
            continue
        else:
            ls=LineString(plst)
            grow = {'date': [dia], 'geometry': [ls]}
            g1=geopandas.GeoDataFrame(grow, crs="EPSG:4326")
            gdf_new=gdf_new.append(g1)
    gdf_new['stroke']="#f71a3a"
    gdf_new['stroke-width']=2
    gdf_new['stroke-opacity']=0.7
    gdf_new.to_file(archivo_salida_gj, driver="GeoJSON")

In [119]:
for i in range(0,182):
    simpl2lines(i)

geolife_000_simpl.geojson  ->  geolife_000_linesimpl.geojson
geolife_001_simpl.geojson  ->  geolife_001_linesimpl.geojson


KeyboardInterrupt: 

In [432]:
df = pd.read_pickle("geolife_106.zip")
df.drop(columns=['time','alt','label','user'],inplace=True)
df['marker-size']="small"
df['marker-color']="small"

In [96]:

dfxb = geopandas.read_file("map.geojson")
dfxb

,marker-color,marker-size,marker-symbol,stroke,stroke-width,stroke-opacity,fill,fill-opacity,geometry
0,#d77070,small,square,None,NaN,NaN,None,NaN,POINT (84.26239 42.79540)
1,None,None,None,#e7dada,2.0,1.0,#ffffff,0.0,"POLYGON ((84.35303 42.86590, 84.33380 42.75710..."
2,None,None,None,None,NaN,NaN,None,NaN,"LINESTRING (84.56039 42.77323, 84.50684 42.899..."


In [97]:
dfxb.to_file("map_write.geojson", driver="GeoJSON")

In [424]:
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.lat, df.lon))